In [ ]:
%pylab inline
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
import matplotlib
from matplotlib import pyplot
import pandas as pd

# 首先是price的qq图

先看一下标准的正态分布

In [ ]:
from scipy import stats
x = np.arange(-5, 5, 0.1)
y = stats.norm.cdf(x, 0, 1)
plt.plot(x, y)

In [ ]:
data = pd.read_csv('winemag-data-130k-v2.csv')
price = data['price']
sorted_ = np.sort(price)
yvals = np.arange(len(sorted_))/float(len(sorted_))
plt.plot(sorted_, yvals)

直观上来看并不符合正态分布，我们看一下qq图

In [ ]:
price.describe()

首先我们把异常值null全部删除

In [ ]:
price_ = price.dropna()

然后绘制price的qq图

In [ ]:
stats.probplot(price_, dist="norm", plot=plt)
plt.show()

可以看到price价格并不符合一个正态分布

# 然后是points的qq图

In [ ]:
points = data['points']
points.describe()

In [ ]:
points.isnull().any()

可见数值属性points不存在空值

In [ ]:
sorted_ = np.sort(points)
yvals = np.arange(len(sorted_))/float(len(sorted_))
plt.plot(sorted_, yvals)

points看起来似乎有一点正态分布的感觉

In [ ]:
stats.probplot(points, dist="norm", plot=plt)
plt.show()

# 看起来points还是比较符合正态分布的

# 2.数据缺失的处理

In [ ]:
# 首先查看price中的缺失数据
data[price.isnull().values==True].head()

从price缺失的数据中我们发现大多是一些数据不完整的红酒，比如没有品酒师或者是产地不详或者是品酒师没有推特的一些记录，缺失的原因可能是市场上已经买不到（下架？绝版？），或者是比较不出名导致没有统计到。

下面对缺失部分使用不同的策略进行处理：

我们引入sklearn库来实现插值策略

In [ ]:
from sklearn.preprocessing import Imputer
import statsmodels.formula.api as smf

In [ ]:
X = data.iloc[:, 4:6].values
X

## 1. 删除缺失数据

In [ ]:
# 首先是删除缺失数据
price_drop = price.dropna()
# 用最高频率值来填补缺失值
imr = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis = 0 )
imr = imr.fit(X)
price_most = imr.transform(X)
price_most[:,-1]

## 2. 用最高频率值来填补缺失值

In [ ]:
type(price)

from pandas import DataFrame,Series
price_m = Series(price_most[:,-1])
type(price_m)

## 3. 通过属性的相关关系来填补缺失值(通过列之间的相关关系）


In [ ]:
# 首先判断一下相关性，价格应该和红酒的得分成正相关的，我们从相关系数来看一下
price.corr(points)

可见价格和红酒评分之间为低度相关，但是关注其他的属性发现相关性更低，所以我们用points来估计缺失的价格

In [ ]:
process_df = data[['price', 'points']]
# 乘客分成已知该特征和未知该特征两部分
known = process_df[process_df.price.notnull()].as_matrix()
unknown = process_df[process_df.price.isnull()].as_matrix()
known[:,1].shape


In [ ]:
import copy
from sklearn.ensemble import RandomForestRegressor
data_Attributes = copy.deepcopy(data)
def setting_null_params(data):
    # 把已有的数值型特征取出来输入到RandomForestRegressor中
    process_df = data[['price', 'points']]
    # 乘客分成已知该特征和未知该特征两部分
    known = process_df[process_df.price.notnull()].as_matrix()
    unknown = process_df[process_df.price.isnull()].as_matrix()
    # X为特征属性值
    X = known[:, 1].reshape(-1,1)
    # y为结果标签值
    y = known[:, 0].reshape(-1,1)
    # fit到RandomForestRegressor之中
    rfr = RandomForestRegressor(random_state=0, n_estimators=2000,  n_jobs=-1)
    rfr.fit(X,y)
    # 用得到的模型进行未知特征值预测
    predicted = rfr.predict(unknown[:, 1].reshape(-1,1))
    # 用得到的预测结果填补原缺失数据
    data_Attributes.loc[(data_Attributes.price.isnull()), 'price'] = predicted
    return data_Attributes

data_Attributes = setting_null_params(data_Attributes)

In [ ]:
data_Attributes['price'].isnull().sum()

**可以看到在price中缺失值已经根据points拟合的函数填充好**

## 4. 通过数据对象之间的相似性来填补缺失值
**通过数据对象之间的相似性来填补缺失值，即通过观察行之间的相似性。我们采用的策略是计算同一酒庄酒类价格的平均追作为该红酒price，如果这个酒庄只有这一个元素，我们使用同一产地下酒类价格平均值**

In [ ]:
data.ix[1]

In [ ]:
winery =  'Quinta dos Avidagos'

data[data['winery']==winery].price.mean() 

In [ ]:
data.ix[0].price.mean()

In [ ]:
price = data['price']
for i in arange(len(price)):
    sum1=0
    if np.isnan(price[i]) == True:
        winery = data.ix[i].winery
        p = data[data['winery']==winery].price.mean()
        if np.isnan(p) == True: 
            region = data.ix[i].region_1
            p =  data[data['region_1']==region].price.mean()
            if np.isnan(p) == True:
                province = data.ix[i].province
                p =  data[data['province']==province].price.mean()
                if np.isnan(p) == True:
                    taster = data.ix[i].taster_name
                    p =  data[data['taster_name']==taster].price.mean()
        price[i] = p

**发现异常值都已经在我们的策略下填充好**

In [ ]:
data['price'].isnull().sum()
#price[price.isnull()]
#data.ix[16749]

# 下面是处理后的数据和原数据的统计对比

## 1. 删除缺失数据的QQ图

In [ ]:
stats.probplot(price_drop, dist="norm", plot=plt)
plt.show()

## 2.用众数补充缺失值的QQ图

In [ ]:
stats.probplot(price_m, dist="norm", plot=plt)
plt.show()

## 3. 通过属性的相关关系来填补缺失值的QQ图

In [ ]:
stats.probplot(data_Attributes['price'], dist="norm", plot=plt)
plt.show()

## 4.通过数据对象之间的相似性来填补缺失值

In [ ]:
stats.probplot(price, dist="norm", plot=plt)
plt.show()